# Colab init

In [2]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Colab Notebooks/IR_Project/'

Mounted at /content/drive


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 50.2 MB/s eta 0:00:00


# Test

In [48]:
from collections import Counter


def font_beautify(font_sizes):
  size_counter = Counter(list(font_sizes))
  body_font_size = size_counter.most_common(1)[0][0]
  font_sizes.discard(body_font_size)
  sorted_sizes = sorted(font_sizes, reverse=True)
  fonts = {size: f"Header {index + 1}" for index, size in enumerate(sorted_sizes)}
  fonts[body_font_size] = "Body"
  return fonts

In [49]:

import pdfplumber

def extract_paragraphs(pdf_path):
    paragraphs = []
    font_sizes = set()
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages[0:2]:
            # Get the text blocks
            for block in page.extract_text_lines():
                    chars = block['chars']
                    font_size = round(chars[0]['size'], 2)
                    font_type = chars[0]['fontname']
                    content = str(block['text'])
                    # print(content.isdigit())
                    if not content.isdigit():
                      font_sizes.add(font_size)
                      paragraphs.append({
                        'text': content,
                        'size': font_size,
                        'font': font_type
                    })




    return paragraphs , font_sizes

pdf_path = folder_path +'Depressive Disorders.pdf'
paragraphs ,font_sizes = extract_paragraphs(pdf_path)
fonts = font_beautify(font_sizes)
print("Unique font sizes:", font_sizes)
# for paragraph in paragraphs:
#     print(f"Text: {paragraph['text']}, Size: {paragraph['size']}, Font: {paragraph['font']}")

Unique font sizes: {12.96, 13.5, 16.98, 22.98}


In [50]:
import json
from collections import defaultdict


# Recursive function to build the JSON-like structure
def group_by_size_recursively(paragraphs, current_size, fonts):
    result = []
    while paragraphs:
        paragraph = paragraphs.pop(0)

        # If the next paragraph has a larger font size, return to the previous level
        if paragraph["size"] > current_size:
            paragraphs.insert(0, paragraph)  # put it back for the caller level
            return result

        # If the next paragraph has the same size, add it directly
        if paragraph["size"] == current_size:
            result.append({"content": fonts[paragraph["size"]], "text": paragraph["text"], "size": paragraph["size"]})
        else:
            # Create a sub-layer for smaller sizes
            paragraphs.insert(0, paragraph)
            sub_layer = group_by_size_recursively(paragraphs, paragraph["size"], fonts)
            result.append({
                "sub_layer": sub_layer
            })

    return result

# Create the layered structure
layered_structure = group_by_size_recursively(paragraphs, paragraphs[0]["size"] ,fonts)

# Convert to JSON
json_output = json.dumps(layered_structure, indent=4)
print(json_output)


[
    {
        "content": "Header 1",
        "text": "Depressive",
        "size": 22.98
    },
    {
        "content": "Header 1",
        "text": "Disorders",
        "size": 22.98
    },
    {
        "sub_layer": [
            {
                "content": "Header 3",
                "text": "Depressive disorders",
                "size": 13.5
            },
            {
                "sub_layer": [
                    {
                        "content": "Body",
                        "text": "include disruptive mood dysregulation disorder, major",
                        "size": 9.48
                    },
                    {
                        "content": "Body",
                        "text": "depressive disorder (including major depressive episode), persistent depressive disorder",
                        "size": 9.48
                    },
                    {
                        "content": "Body",
                        "text": "(dysthymia), premenstrual d

# Old Code

In [ ]:
import pdfplumber

pdf_file_name = folder_path +'Depressive Disorders.pdf'
lines = []


with pdfplumber.open(pdf_file_name) as pdf:
    text = pdf.pages[0].extract_text()
    print(text)
    for page in pdf.pages:
        current_line = []
        words = page.extract_words()

        for i in range(len(words)):
            current_line.append(words[i])

            if i == len(words) - 1:
                lines.append(current_line)
                current_line = []
                break

            if abs(words[i + 1]["top"] - words[i]["top"]) > 4.5:
                lines.append(current_line)
                current_line = []

        lines.append("*") # end of page


Depressive
Disorders
Depressive disorders
include disruptive mood dysregulation disorder, major
depressive disorder (including major depressive episode), persistent depressive disorder
(dysthymia), premenstrual dysphoric disorder, substance/medication-induced depres-
sive disorder, depressive disorder due to another medical condition, other specified de-
pressive disorder, and unspecified depressive disorder. Unlike in DSM-IV, this chapter
“Depressive Disorders” has been separated from the previous chapter “Bipolar and Re-
lated Disorders.” The common feature of all of these disorders is the presence of sad,
empty, or irritable mood, accompanied by somatic and cognitive changes that signifi-
cantly affect the individual’s capacity to function. What differs among them are issues of
duration, timing, or presumed etiology.
In order to address concerns about the potential for the overdiagnosis of and treatment
for bipolar disorder in children, a new diagnosis, disruptive mood dysregulation

In [19]:
from collections import Counter

def classify_text_by_size(font_data):
    sizes = [item['size'] for item in font_data]

    # Determine a threshold for header vs. body size
    size_counter = Counter(sizes)
    common_sizes = size_counter.most_common()

    # Assuming headers use the largest font size
    header_size = common_sizes[0][0]  # largest font size
    body_size = common_sizes[-1][0]  # smallest or next common size

    headers = [item['text'] for item in font_data if item['size'] == header_size]
    body_text = [item['text'] for item in font_data if item['size'] == body_size]

    return headers, body_text

headers, body_text = classify_text_by_size(paragraphs)
print("Headers:", headers)
print("Body Text:", body_text)


Headers: ['include disruptive mood dysregulation disorder, major', 'depressive disorder (including major depressive episode), persistent depressive disorder', '(dysthymia), premenstrual dysphoric disorder, substance/medication-induced depres-', 'sive disorder, depressive disorder due to another medical condition, other specified de-', 'pressive disorder, and unspecified depressive disorder. Unlike in DSM-IV, this chapter', '“Depressive Disorders” has been separated from the previous chapter “Bipolar and Re-', 'lated Disorders.” The common feature of all of these disorders is the presence of sad,', 'empty, or irritable mood, accompanied by somatic and cognitive changes that signifi-', 'cantly affect the individual’s capacity to function. What differs among them are issues of', 'duration, timing, or presumed etiology.', 'In order to address concerns about the potential for the overdiagnosis of and treatment', 'for bipolar disorder in children, a new diagnosis, disruptive mood dysregulati

In [ ]:
def check_and_remove(i) :
    for w in lines[i]:
        if w["text"].isdigit() and lines[i][-1]["text"][-1] != '.':
            lines[i] = '*'
            return True

star_indexes = [-1] + [i for i, v in enumerate(lines) if v == '*']

for first,last in zip(star_indexes, star_indexes[1:]):
    if not check_and_remove(first + 1):
        check_and_remove(last - 1)

for i in range(lines.count('*')):
    lines.remove('*')

In [ ]:
parts = [[]]

for line in lines:
    if len(line) < 5 and line[-1]["text"][-1] != '.':
        parts[-1].append(line)
        parts.append([])

    else:
        parts[-1].append(line)

        if line[-1]["text"][-1] == '.':
            parts.append([])


In [ ]:
for part in parts:
    for line in part:
        for w in line:
            print(w['text'], end=" ")
        print()
    print()
    print()


Depressive 


Disorders 


Depressive disorders include disruptive mood dysregulation disorder, major 
depressive disorder (including major depressive episode), persistent depressive disorder 
(dysthymia), premenstrual dysphoric disorder, substance/medication-induced depres- 
sive disorder, depressive disorder due to another medical condition, other specified de- 
pressive disorder, and unspecified depressive disorder. Unlike in DSM-IV, this chapter 
“Depressive Disorders” has been separated from the previous chapter “Bipolar and Re- 
lated Disorders.” The common feature of all of these disorders is the presence of sad, 
empty, or irritable mood, accompanied by somatic and cognitive changes that signifi- 
cantly affect the individual’s capacity to function. What differs among them are issues of 
duration, timing, or presumed etiology. 


In order to address concerns about the potential for the overdiagnosis of and treatment 
for bipolar disorder in children, a new diagnosis, disruptive

In [ ]:
with pdfplumber.open(pdf_file_name) as pdf:

    fonts = []
    for page in pdf.pages:
        words = page.extract_words()

        for i in range(len(words)):
            if not words[i]["height"] in fonts:
                fonts.append(words[i]["height"])

fonts.sort()
print(fonts)

[8.459999999999923, 8.45999999999998, 8.460000000000036, 9.479999999999905, 9.479999999999961, 9.480000000000018, 9.480799999999931, 9.50843999999995, 9.508440000000007, 9.508440000000064, 9.527399999999943, 9.55583999999999, 10.678799999999967, 11.747919999999965, 12.959999999999923, 12.95999999999998, 12.960000000000036, 13.499999999999943, 13.5, 13.500000000000057, 16.980000000000018, 22.980000000000018]
